<a href="https://colab.research.google.com/github/yongsa-nut/SF323_CN408_AIEngineer/blob/main/SF323_CN408_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW 2 (5 points)

## 1 Count Tokens (1 point)

Count the number of tokens using [gpt-4o tokenizer](https://platform.openai.com/tokenizer) (you can use the web or the code) of the following four prompts:

1. The old man sat on the park bench, feeding pigeons and lost in thought. As the sun began to set, he smiled, realizing that sometimes the simplest moments are the most precious.

2. ชายชรานั่งอยู่บนม้านั่งในสวนสาธารณะ ให้อาหารนกพิราบและจมอยู่ในภวังค์ความคิด เม่ือดวงอาทิตย์เรี่มตกดิน เขายิ้ม ตระหนักว่าบางครั้งช่วงเวลาที่เรียบง่ายที่สุดกลับมีค่าที่สุด

3. 老人は公園のベンチに座り、鳩に餌をやりながら物思いにふけっていた。日が沈み始めると、彼は微笑み、時として最も単純な瞬間が最も貴重であることに気づいた。

4. 9.11 > 9.9

**Answer**:

1.

2.

3.

4.

## 2) Word Embedding: Closet words (1 point)

For this question, your job is find the closest words of the following words in the vector space of Word2Vec embedding.

1) cat

2) King

3) teaching

4) learning

The code is provided below.

In [ ]:
# First install the library spacy
!pip install gensim

In [ ]:
import gensim.downloader as api
from gensim.models import KeyedVectors
import numpy as np

# Load pre-trained Word2Vec embeddings (~1.6gb)
model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# Example of usage
word = 'Merry'
model.similar_by_word(word) # It will return top 10 closest words. ['word','cosine similarity']

[('Shavaun', 0.49784329533576965),
 ('Apple_Dumpling', 0.4907097816467285),
 ('Merry_Merry', 0.4760850667953491),
 ('Christma', 0.4625037908554077),
 ('Gravener', 0.4579834043979645),
 ('OPEN_RESTRICTED_1', 0.45711594820022583),
 ('Reindeers', 0.4541266858577728),
 ('Rub_Dub_Dub', 0.4538874328136444),
 ('Fancy_Nancy_Splendiferous', 0.4518665373325348),
 ('XMas', 0.45152536034584045)]

**Answer**:

1.

2.

3.

4.

## 3) Sentence Embedding: Closest sentences (1 points)

For this question, you will be using cohere API to explore Sentence Embedding. The idea is still the same, similar sentences will be closer in the embedding space.

To run the code below, the first step you must obtain cohere API trial (free) key from their [website](https://dashboard.cohere.ai/welcome/register).

![key_trial.png](https://drive.google.com/uc?export=view&id=1rdMaMdO5S7eaktN0qwodhgQd3auuZ6gi)

* Note: Be careful about rate limit for free key. 5 calls per minute.

In [ ]:
!pip install cohere

In [ ]:
import numpy as np
import cohere
co = cohere.Client("YOUR_COHERE_API_KEY") # Your Cohere API key

In [ ]:
# This is our documents. It is just a list of strings (sentences)
# (Taken from https://github.com/anthropics/anthropic-cookbook/blob/main/third_party/VoyageAI/how_to_create_embeddings.md)
documents = [
    "The Mediterranean diet emphasizes fish, olive oil, and vegetables, believed to reduce chronic diseases.",
    "Photosynthesis in plants converts light energy into glucose and produces essential oxygen.",
    "20th-century innovations, from radios to smartphones, centered on electronic advancements.",
    "Rivers provide water, irrigation, and habitat for aquatic species, vital for ecosystems.",
    "Apple’s conference call to discuss fourth fiscal quarter results and business updates is scheduled for Thursday, November 2, 2023 at 2:00 p.m. PT / 5:00 p.m. ET.",
    "Shakespeare's works, like 'Hamlet' and 'A Midsummer Night's Dream,' endure in literature."
]

# Embedded (More info: https://docs.cohere.com/reference/embed?ref=cohere-ai.ghost.io )
response = co.embed(
    texts=documents,
    model='embed-english-v3.0',
    input_type='search_document'
)

embeddings = response.embeddings

print(f"The dimension of the embedding =  {len(embeddings[0])}")

print(f"\nEmbedding for sentence 1: {embeddings[0]}")
print(f"Embedding for sentence 2: {embeddings[1]}")
print(f"Embedding for sentence 3: {embeddings[0]}")

# To calculate similarity we can use dot product (same as cosine similarity when vectors have been normarlized to 1.)
print("\nSimilarity between sentences 1 and 2:", np.dot(embeddings[0], embeddings[1]))
print("Similarity between sentences 1 and 3:", np.dot(embeddings[0], embeddings[2]))
print("Similarity between sentences 2 and 3:", np.dot(embeddings[0], embeddings[3]))

In [ ]:
# Helper function to find the closest document
def closest_document(sentence, documents, model='embed-english-v3.0'):
    # Merge into one so that it would only use one call to API.
    documents = documents.copy() + [sentence]
    response = co.embed(
        texts=documents,
        model=model,
        input_type='search_document'
    )
    embeddings = response.embeddings
    # Calculate dot products of the sentence and every document
    similarities = [np.dot(embeddings[i], embeddings[-1]) for i in range(len(embeddings)-1)]
    # print similarities and index
    for i, sim in enumerate(similarities):
        print(f"Similarity between sentence {i+1} and the new sentence: {sim}")

    max_index = np.argmax(similarities)
    return max_index, documents[max_index]

In [ ]:
# A similar sentence to the first sentence
test_sentence = "Focusing on seafood, olive oil, and plant-based foods, the Mediterranean eating pattern is thought to lower the risk of long-term health conditions."

index, sentence = closest_document(test_sentence, documents)
print(f"The closest sentence to '{test_sentence}' is '{sentence}' at index {index}.")

**Q3.1** Come up with a sentence that is closest to the fifth sentence (```"Apple’s conference call to discuss fourth fiscal quarter results and business updates is scheduled for Thursday, November 2, 2023 at 2:00 p.m. PT / 5:00 p.m. ET."```)

In [ ]:
your_sentence = "" #Your sentence here

index, sentence = closest_document(your_sentence, documents)
print(f"The closest sentence to '{your_sentence}' is '{sentence}' at index {index}.")

**Q3.2** Come up with a sentence that is closest to the sixth sentence (`"Shakespeare's works, like 'Hamlet' and 'A Midsummer Night's Dream,' endure in literature."`).

Note your sentence must not contain the word 'Shakespeare'

In [ ]:
your_sentence = "" #Your sentence here

index, sentence = closest_document(your_sentence, documents)
print(f"The closest sentence to '{your_sentence}' is '{sentence}' at index {index}.")

**Q3.3** Come up with a sentence in "Thai" that is closest to the second sentence. (`"Photosynthesis in plants converts light energy into glucose and produces essential oxygen."`)

In [ ]:
your_sentence = "" #Your sentence here

index, sentence = closest_document(your_sentence, documents, 'embed-multilingual-v3.0')
print(f"The closest sentence to '{your_sentence}' is '{sentence}' at index {index}.")

**Q3.4** Come up with a sentence in "Japanese" that is closest to the first sentence. (`"The Mediterranean diet emphasizes fish, olive oil, and vegetables, believed to reduce chronic diseases."`)


In [ ]:
your_sentence = "" #Your sentence here

index, sentence = closest_document(your_sentence, documents, 'embed-multilingual-v3.0')
print(f"The closest sentence to '{your_sentence}' is '{sentence}' at index {index}.")

## 4) Multi turn chatbot (1 point)

For this question, your task is to implement a simple multi-turn chatbot by filling in the code **using provided helper functions**

We will be using `gemini-2.5-flash-lite` for this homework (See available models [here](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions)).

[Documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/start/openai) for using Gemini with OpenAI library

### OpenRouter

In [1]:
from google.colab import userdata
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=userdata.get('openrouter'),
)

<br>

---

In [ ]:
# Check that you successfully connect to vertex
response = client.chat.completions.create(
  model="google/gemini-2.5-flash-lite",
  messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": "Hello, test"}
  ]
)

print(response.choices[0].message.content)

In [ ]:
model = "google/gemini-2.5-flash-lite"

# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)

def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)

def chat(messages, temperature=0.7):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=10000,
        temperature = temperature
    )

    return response.choices[0].message.content

Use **provided helper functions** to implement a multi turn chatbot

**Note**: Test your program so I can check that it works!

In [ ]:
# Start with an empty message list
messages = []

while True:
    # Get user input
    user_input = input('User: ')

    # If user types 'quit', break out of the loop
    if user_input == 'quit':
        break

    # Add user input to the list of messages with the helper function
    _______________________________________

    # Call the model with the helper function
    _______________________________________

    # Add model's response to the list of messages with the helper function
    _______________________________________

    # Print the generated text
    _______________________________________

## 5) Prompt Engineering Basic (1 points)

In [ ]:
# Helper function. DO NOT CHANGE. (Must run the earlier cell first)
# Temperature is set to 0 so that the answer will be the same.
def get_response(user_prompt, system_prompt="",temperature=0):
  temp_messages =[
        {'role':'system','content':system_prompt},
        {"role": "user", "content": user_prompt},
  ]

  return chat(temp_messages, temperature = 0)

# Testing if your api key is working
print(get_response("Hello"))

Hello! How can I help you today?


**Q5.1** Get the model to count to three

In [ ]:
prompt = "" #Fill in your prompt here
print(get_response(prompt))

One, two, three.


**Q5.2** Modify the system prompt to make **the model respond like it's a 3 year old child**.

In [ ]:
system_prompt = "" #Fill in your prompt here
prompt = "How big is the sky?"
print(get_response(prompt, system_prompt))

The "sky" is a fascinating concept because its "size" depends entirely on how you define it! Here are a few ways to think about it:

1.  **The Earth's Atmosphere (The Sky We Breathe and See Clouds In):**
    *   This is the layer of gases surrounding our planet. Most of what we consider "sky" in our daily lives (weather, clouds, blue color) happens in the lowest layers.
    *   **Troposphere:** Where most weather occurs, extends up to about 8-15 km (5-9 miles) high.
    *   **Stratosphere:** Contains the ozone layer, extends up to about 50 km (31 miles).
    *   **Mesosphere:** Extends up to about 85 km (53 miles).
    *   **Thermosphere:** Where the aurora occur, extends up to about 600 km (370 miles) or more.
    *   **Exosphere:** The outermost layer, gradually thinning out into space, extending up to about 10,000 km (6,200 miles).
    *   **The Kármán Line:** This is an internationally recognized boundary at **100 km (62 miles)** above Earth's mean sea level, often considered the e

**Q5.3** Adapt the system prompt to make the model output its answer in Thai.


In [ ]:
system_prompt = "" #Fill in your prompt here
prompt = "Hello, how are you?"
print(get_response(prompt, system_prompt))

Hello! I'm functioning perfectly and ready to assist you. Thank you for asking!

How can I help you today?


**Q5.4** Modify the basketball player prompt so that Claude doesn't equivocate at all and responds with **ONLY the name of one specific player**, with no other words or punctuation.


In [ ]:
prompt = "Who is the best basketball player of all time?"
print(get_response(prompt))

That's one of the most debated questions in sports, and there's no single definitive answer! It largely depends on what criteria you value most.

However, the two players most consistently at the center of the "Greatest Of All Time" (GOAT) debate are:

1.  **Michael Jordan:**
    *   **Arguments for:** Unblemished 6-0 record in the NBA Finals, 6 Finals MVPs, 5 regular season MVPs, 10 scoring titles, Defensive Player of the Year award, unparalleled clutch performance, incredible competitive drive, and global cultural impact. Many believe he was the most dominant player in his era and had a perfect "peak."
    *   **Arguments against:** Played in a different era with different rules (e.g., hand-checking), shorter career at the absolute elite level compared to some others.

2.  **LeBron James:**
    *   **Arguments for:** Unmatched longevity and sustained excellence (still playing at an elite level in his 20th+ season), all-time leading scorer in NBA history, 4 NBA championships with 3 di

**Q5.5** Modify the prompt so that Gemini responds with as long a response as you can muster. Gemini's response should be over 800 words.


In [ ]:
prompt = "Write me a short story?"
response = get_response(prompt)
print(f"Number of words = {len(response.split())}")
print(response)

Number of words = 472
Elara loved the quiet hum of the old city library. It was a place of hushed reverence, where the scent of aged paper and dust motes dancing in sunbeams felt like a comforting embrace. She wasn't looking for anything in particular today, just wandering the labyrinthine aisles, her fingers trailing over the spines of forgotten tales.

Deep in a rarely visited section, tucked behind a row of weighty tomes on ancient cartography, she found it. It wasn't grand or ornate, just a slim, unassuming book with a cover of smooth, dark wood, no title, no author. It felt cool to the touch, almost alive.

Curiosity piqued, Elara pulled it free and settled into a worn armchair by a tall window. She opened the book. The pages were utterly blank. Not yellowed or stained, but a pristine, almost luminous white. Disappointment pricked at her, but as she was about to close it, a faint shimmer, like heat haze over a summer road, rippled across the first page.

Then, slowly, ink began to